In [1]:
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
tqdm.pandas()

import pandas as pd

In [2]:
from utils import filter_by_media
from utils import cluster_by_month
#from utils import preprocess


df = pd.read_csv("data/loslagos-comunas.csv")[:1000]
df = cluster_by_month(filter_by_media(df))
#df['tokens'] =  df.content.progress_apply(lambda x: preprocess(str(x)))
df.isna().any()

date               False
media_outlet       False
url                False
title              False
text               False
content            False
comuna              True
date_clustering    False
dtype: bool

## 1. Modelado de tópicos con BERTopic

In [3]:
docs = df.content.tolist()

In [4]:
from bertopic import BERTopic
import spacy
nlp = spacy.load("es_core_news_md")

c:\users\56981\appdata\local\programs\python\python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df['topic'] = ""

topic_model = BERTopic(language="multilingual",
                       embedding_model=nlp,
                       calculate_probabilities=True,
                       verbose=True)

topics, probs = topic_model.fit_transform(docs)

labels=topic_model.generate_topic_labels()
count = 0
for doc in docs:  
    df.at[df.index[df['content'] == doc], 'topic'] = labels[topics[count]+1]
    count+=1
topic_model.get_topic_info()

100%|████████████████████████████████████████████████████████████████████████████████| 990/990 [00:32<00:00, 30.49it/s]
2022-10-24 01:17:56,612 - BERTopic - Transformed documents to Embeddings
2022-10-24 01:18:08,092 - BERTopic - Reduced dimensionality
2022-10-24 01:18:08,259 - BERTopic - Clustered reduced embeddings


,Topic,Count,Name
0,-1,249,-1_regional_puerto_región_chile
1,0,133,0_colo_equipo_jugadores_puntos
2,1,49,1_presidente_presidencial_retiro_piñera
3,2,43,2_víctima_pareja_hombre_mujer
4,3,43,3_chiloé_alcalde_plan_alcaldes
5,4,36,4_canción_disco_vida_cantante
6,5,33,5_mil_millones_inversión_presupuesto
7,6,32,6_salud_alimentación_física_peso
8,7,30,7_casos_nuevos_contagios_total
9,8,28,8_mujeres_pesca_artesanal_cursos


In [15]:
df.loc[df['topic'] == '9_cultural_artes_cultura'][:3]

,date,media_outlet,url,title,text,content,comuna,date_clustering,topic
4,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/galeria-de-ar...,Galería de Arte Machacoya realizará remate de ...,"Hoy viernes a las 18:30 horas, en Machacoya At...",galería arte machacoya realizará remate obras ...,NaN,2021-10,9_cultural_artes_cultura
20,2021-10-01,elquellonino,https://elquellonino.cl/29949/festival-cultura...,Festival “Cultura por Fibra” y II Encuentro Pr...,El Centro Cultural Municipal de Castro también...,festival “ cultura fibra ” ii encuentro provin...,['castro'],2021-10,9_cultural_artes_cultura
123,2021-10-01,elinsular,https://elinsular.cl/fogon-cultural/2021/10/01...,Dos consideraciones sobre el “Premio Chiloé de...,Interesante y laudable galardón anual entregad...,dos consideraciones “ premio chiloé extensión ...,['castro'],2021-10,9_cultural_artes_cultura


In [19]:
df.loc[df['topic'] == '0_colo_equipo_jugadores']

,date,media_outlet,url,title,text,content,comuna,date_clustering,topic
53,2021-10-01,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/1...,El gol no quiso nada con Colo Colo y su derrot...,Sorpresa total en el estadio Monumental. No hu...,gol quiso colo colo derrota deja torneo rojo s...,NaN,2021-10,0_colo_equipo_jugadores
54,2021-10-01,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/1...,La U. de Chile le ofrece renovación por un año...,Ayer era el plazo fatal autoimpuesto por la di...,u chile ofrece renovación año larrivey arias a...,NaN,2021-10,0_colo_equipo_jugadores
55,2021-10-01,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/1...,"UC y el duelo con La Calera de mañana: ""Hay qu...",11 goles en tres partidos suma Universidad Cat...,uc duelo calera mañana ganar 11 goles tres par...,NaN,2021-10,0_colo_equipo_jugadores
57,2021-10-01,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/1...,Chile definía su lugar en Mundial Sub 16 de Te...,"Desde las 5:00 a. m. de Chile, los equipos mas...",chile definía lugar mundial sub 16 tenis madru...,NaN,2021-10,0_colo_equipo_jugadores
58,2021-10-01,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/1...,Cristian Garin estrenó colaboración con Jorge ...,Juan Carlos Sáez compartió una foto en Crandon...,cristian garin estrenó colaboración jorge agui...,NaN,2021-10,0_colo_equipo_jugadores
...,...,...,...,...,...,...,...,...,...
991,2021-10-07,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/1...,Chile apostará por inédita fórmula ofensiva co...,"La ""Operación Remontada"", como calificó Franci...",chile apostará inédita fórmula ofensiva cuatro...,NaN,2021-10,0_colo_equipo_jugadores
993,2021-10-07,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/1...,"""Tenemos que retomar lo que éramos antes del S...",Son doce los puntos que separan a la Universid...,retomar superclásico doce puntos separan univ...,NaN,2021-10,0_colo_equipo_jugadores
994,2021-10-07,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/1...,Arturo Vidal y Ángelo Sagal se quedaron en San...,Si bien el plantel de Chileviajó ayer a Perú p...,arturo vidal ángelo sagal quedaron santiago si...,NaN,2021-10,0_colo_equipo_jugadores
996,2021-10-07,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/1...,La Ruca alba espera 30 mil hinchas el sábado,Dada la nueva realidad de la pandemia y la mod...,ruca alba espera 30 mil hinchas sábado dada nu...,NaN,2021-10,0_colo_equipo_jugadores


## 2. Análisis de sentimiento 

In [ ]:
#!pip install pysentimiento

In [20]:
sub = df.copy()
sub['title_sentiment_roBERTuito'] = ""
sub['title_emotion_roBERTuito'] = ""
sub['title_sentiment_BETO'] = ""
sub['text_sentiment_BETO'] = ""

In [ ]:
# roBERTuito
from pysentimiento import create_analyzer
sentiment_analyzer = create_analyzer(task="sentiment", lang="es")
emotion_analyzer = create_analyzer(task="emotion", lang="es")

In [ ]:
# BETO
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
model_name = "finiteautomata/beto-sentiment-analysis"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [23]:
for index, row in tqdm(sub.iterrows(), desc='sub rows - sentiment', total=sub.shape[0]):
    # análisis del título de la noticia
    sub.at[index, "title_sentiment_roBERTuito"] = sentiment_analyzer.predict(row['title'])
    sub.at[index, "title_emotion_roBERTuito"] = emotion_analyzer.predict(row['title'])
    sub.at[index, 'title_sentiment_BETO'] = nlp(row['title'])
    
    # análisis del cuerpo de la noticia
    count_neutral = 0
    count_negative = 0
    count_positive = 0
    partition = row['text'].split(".")
    for text in partition:
        # Analizamos su sentimiento
        sentiment_value = nlp(text)
        if sentiment_value[0].get('label') == "NEU": count_neutral=count_neutral+1
        if sentiment_value[0].get('label') == "NEG": count_negative=count_negative+1
        if sentiment_value[0].get('label') == "POS": count_positive=count_positive+1
            
    sub.at[index, "text_sentiment_BETO"] = {"NEU": count_neutral, "NEG": count_negative, "POS": count_positive}

sub rows - sentiment: 100%|██████████████████████████████████████████████████████████| 990/990 [18:35<00:00,  1.13s/it]


In [24]:
pd.set_option("display.max_rows", None, "display.max_columns", None, 'display.max_colwidth', None)
sub[['title','title_sentiment_roBERTuito', 'title_emotion_roBERTuito','title_sentiment_BETO',"text_sentiment_BETO"]]

,title,title_sentiment_roBERTuito,title_emotion_roBERTuito,title_sentiment_BETO,text_sentiment_BETO
0,Reconocen a guardaparques de la Región de Los Lagos como actores claves en la conservación,"AnalyzerOutput(output=NEU, probas={NEU: 0.627, POS: 0.355, NEG: 0.018})","AnalyzerOutput(output=others, probas={others: 0.855, joy: 0.103, surprise: 0.014, sadness: 0.009, anger: 0.009, disgust: 0.005, fear: 0.005})","[{'label': 'POS', 'score': 0.9539938569068909}]","{'NEU': 13, 'NEG': 0, 'POS': 14}"
1,Con nuevos materiales comienza plan piloto en Saltos del Petrohué,"AnalyzerOutput(output=NEU, probas={NEU: 0.853, POS: 0.136, NEG: 0.010})","AnalyzerOutput(output=others, probas={others: 0.951, joy: 0.021, surprise: 0.013, fear: 0.005, sadness: 0.004, anger: 0.003, disgust: 0.002})","[{'label': 'NEU', 'score': 0.9277083277702332}]","{'NEU': 3, 'NEG': 0, 'POS': 6}"
2,Centro de Salud Familiar CESFAM Puerto Varas invita a prevenir el Cáncer Cervicouterino con extensión horaria para exámenes PAP,"AnalyzerOutput(output=NEU, probas={NEU: 0.954, POS: 0.026, NEG: 0.020})","AnalyzerOutput(output=others, probas={others: 0.940, joy: 0.030, anger: 0.008, surprise: 0.008, sadness: 0.007, fear: 0.004, disgust: 0.003})","[{'label': 'NEU', 'score': 0.9948523640632629}]","{'NEU': 14, 'NEG': 0, 'POS': 0}"
3,Alcalde Tomás Gárate presidió por primera vez la octava sesión del Consejo Comunal de la Sociedad Civil COSOC Puerto Varas,"AnalyzerOutput(output=NEU, probas={NEU: 0.916, POS: 0.052, NEG: 0.032})","AnalyzerOutput(output=others, probas={others: 0.933, joy: 0.048, surprise: 0.008, anger: 0.004, sadness: 0.003, fear: 0.002, disgust: 0.002})","[{'label': 'NEU', 'score': 0.9440039396286011}]","{'NEU': 8, 'NEG': 0, 'POS': 0}"
4,Galería de Arte Machacoya realizará remate de obras de artistas de la zona,"AnalyzerOutput(output=NEU, probas={NEU: 0.831, POS: 0.158, NEG: 0.011})","AnalyzerOutput(output=others, probas={others: 0.940, joy: 0.034, sadness: 0.009, surprise: 0.008, fear: 0.004, anger: 0.004, disgust: 0.002})","[{'label': 'NEU', 'score': 0.9955322742462158}]","{'NEU': 3, 'NEG': 0, 'POS': 4}"
5,Municipio llamará a licitación construcción de pasarela peatonal en Los Notros,"AnalyzerOutput(output=NEU, probas={NEU: 0.947, POS: 0.037, NEG: 0.016})","AnalyzerOutput(output=others, probas={others: 0.948, joy: 0.017, surprise: 0.016, sadness: 0.006, fear: 0.005, anger: 0.005, disgust: 0.003})","[{'label': 'NEU', 'score': 0.9952103495597839}]","{'NEU': 5, 'NEG': 0, 'POS': 1}"
6,3era Mesa de Reactivación Económica circunscribe su radio de acción,"AnalyzerOutput(output=NEU, probas={NEU: 0.978, NEG: 0.012, POS: 0.010})","AnalyzerOutput(output=others, probas={others: 0.979, surprise: 0.008, joy: 0.007, fear: 0.002, sadness: 0.001, anger: 0.001, disgust: 0.001})","[{'label': 'NEU', 'score': 0.9724546670913696}]","{'NEU': 12, 'NEG': 0, 'POS': 1}"
7,Gremio médico rechaza cierre de camas críticas implementadas en Hospital Ancud,"AnalyzerOutput(output=NEU, probas={NEU: 0.878, NEG: 0.113, POS: 0.009})","AnalyzerOutput(output=others, probas={others: 0.949, sadness: 0.016, surprise: 0.015, anger: 0.007, fear: 0.005, disgust: 0.004, joy: 0.004})","[{'label': 'NEU', 'score': 0.9739839434623718}]","{'NEU': 2, 'NEG': 2, 'POS': 1}"
8,Asistentes de educación municipal en Osorno verán incrementadas sus remuneraciones,"AnalyzerOutput(output=NEU, probas={NEU: 0.929, NEG: 0.062, POS: 0.009})","AnalyzerOutput(output=others, probas={others: 0.954, surprise: 0.014, sadness: 0.013, anger: 0.007, fear: 0.005, joy: 0.005, disgust: 0.003})","[{'label': 'NEU', 'score': 0.9927809834480286}]","{'NEU': 4, 'NEG': 0, 'POS': 4}"
9,CONADI llama a renovar directivas de comunidades indígenas,"AnalyzerOutput(output=NEU, probas={NEU: 0.916, NEG: 0.060, POS: 0.024})","AnalyzerOutput(output=others, probas={others: 0.971, surprise: 0.007, joy: 0.006, anger: 0.005, sadness: 0.005, fear: 0.003, disgust: 0.002})","[{'label': 'NEU', 'score': 0.9907516241073608}]","{'NEU': 11, 'NEG': 0, 'POS': 0}"
